In [ ]:
# Spring 2022
# loading images

In [10]:
import numpy as np
import matplotlib.pyplot as plt 
import model
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
%matplotlib inline

use_gpu = torch.cuda.is_available()

In [11]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

class NPY_dataset(torch.utils.data.Dataset):
    def __init__(self, npy_path, transform=None):
        self.npy_path = npy_path
        self.npy_data = np.load(self.npy_path)
        self.transform = transform
    def __len__(self):
        return len(self.npy_data)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = self.npy_data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

In [12]:
class Net(nn.Module): ## input shape: (batch_size, 150, 150, 1) output shape: (1, 4)
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 4)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    

In [13]:
x_array = np.array(np.load('X_train.npy')).reshape(-1, 150, 150, 1)
print(x_array.shape)
X_train = torch.from_numpy(x_array)
y_data_ori = np.load('y_train.npy')
y_data = []
for item in y_data_ori:
    binary = [0, 0, 0, 0]
    binary[item] = 1
    y_data.append(binary)
y_train = torch.from_numpy(np.array(y_data))
# y_train = torch.from_numpy(np.array(np.load('y_train.npy').astype(np.int32)))
# dataset_input = NPY_dataset('X_train.npy', transform=transform)
# dataset_label = NPY_dataset('y_train.npy', transform=transform)
dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)


(3320, 150, 150, 1)


In [14]:
net = Net()
if use_gpu:
    net = net.cuda()
print(net)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(100):  # loop over the dataset multiple times
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if use_gpu:
            inputs = inputs.cuda()
            labels = labels.cuda()
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if i + 1 % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, loss.item()))
    if epoch % 10 == 0:
        print('epoch: %d, loss: %.3f' % (epoch + 1, loss.item()))
    if epoch % 50 == 0:
        torch.save(net.state_dict(), 'model.pth')

print('Finished Training')

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout2d(p=0.25, inplace=False)
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=4, bias=True)
)


RuntimeError: Given groups=1, weight of size [32, 1, 3, 3], expected input[64, 150, 150, 1] to have 1 channels, but got 150 channels instead

In [ ]:
# load the raw data
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
print('shape of the training data:', X_train.shape)
print('shape of the target data:', y_train.shape)

In [ ]:
# preview some of the training images
plt.figure(figsize=(12,10))
x, y =3, 3
for i in range(9):  
    plt.subplot(y, x, i+1)
    plt.imshow(X_train[i,:,:],cmap='gray')
    plt.title('label: {}'.format(y_train[i]))
    plt.axis('off')
plt.show()

In [ ]:
# analyze data

In [ ]:
# train you model

In [ ]:
# load the test data
X_test = np.load('X_test.npy')
print('shape of the test data:', X_test.shape)

In [ ]:
# make prediction for the test data